## Read full CSV 

Place this notebook inside the data folder so all paths work!

In [1]:
import pandas as pd
%pip install music21
from music21 import *
#This is the path to musescore.exe
path_to_musescore = 'C:/Users/ifc04/OneDrive/Desktop/UNIVERSIDAD/Audio and Music Processing Lab/Lab 3 - Ethnomusicology/GMD-classification/data/'
environment.set('musescoreDirectPNGPath', path_to_musescore)


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: C:\Users\ifc04\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:

#Read galician_musicxml_full.csv but is in father folder
df = pd.read_csv('../testing/galician_musicxml_full.csv',quotechar='"',skipinitialspace=True)

#df = pd.read_csv('galician_musicxml_full.csv',quotechar='"',skipinitialspace=True)
#name first column id
df = df.rename(columns={'Unnamed: 0': 'id'})
#In this case we dont need the first column because index of row=name of musicxml file
#have in mind that we do need it for the other csv files
df

,id,name
0,0,Muiñeira dos Fosados
1,1,Detrás das airas
2,2,Muiñeira de Quindous
3,3,Muiñeira de Quindous
4,4,Muiñeira
...,...,...
448,448,Carballesa
449,449,Carballesa do Incio
450,450,Muiñeira de Vilarmide
451,451,Na Ruada


## Read file with all the locations

In [3]:

#Read galician_musicxml_full.csv in father folder
df_located = pd.read_csv('../testing/galician_musicxml_located.csv',quotechar='"',skipinitialspace=True)
#df_located = pd.read_csv('galician_musicxml_located.csv',quotechar='"',skipinitialspace=True)
#Rename first column to id
df_located = df_located.rename(columns={df_located.columns[0]: "id"})
"""
Have in mind that for this case the id IS important. ID is the name of the musicxml file
"""
df_located


,id,name,parroquia,concellu,province
0,1,Detrás das airas,NaN,As Airas,Ourense
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo
3,5,Muiñeira de Monforte,NaN,Monforte,Lugo
4,6,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra
...,...,...,...,...,...
253,446,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra
254,449,Carballesa do Incio,NaN,O Incio,Lugo
255,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo
256,452,Danza Anterga de Lugo,NaN,Lugo,Lugo


In [4]:
#Example, let's say we want to load the Muiñeira de Vilarmide	
#Show all the rows where the id is equal to "Muiñeira de Quindous"
df_located.loc[df_located['name'] == 'Muiñeira de Vilarmide']


,id,name,parroquia,concellu,province
255,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo


## Time signature
We get this data from music21 and save it to our dataframe.

In [5]:
def load_musicxml(index):
    """
    Load a musicxml file and return a music21 stream
    """
    #Set musicxml_file as a file in path data/name and name being string of the index and .musicxml
    #musicxml_file = 'data/' + str(index) + '.musicxml'
    musicxml_file =str(index) + '.musicxml'
    #The files are in the folder data
    #Load the musicxml file
    #print(musicxml_file)
    #try:
    #    s = corpus.parse(musicxml_file)

    #except Exception as e:
        #Get exception if the file is not found
    s=converter.parse(musicxml_file)
    #    return s
    return s

#Let's continue with the example, loading hte second muiñeira that appears:
#muiñeira=load_musicxml(3)

#For loop to 453
#Create dataframe with id, name, key
df_key=pd.DataFrame(columns=['id','name','key','time_signature'])
for i in range(453):
    #get index of df_located
    index=df.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
         
        key=s.analyze('key')
        #Get name of the song in df in this case we can use the index because the index of df is the same as the id
        name=df.loc[i]['name']
        measures0 = s.getElementsByClass(stream.Measure).stream() #numero de compases
        tS = measures0[0].getElementsByClass(meter.TimeSignature).stream()[0]

        time_signature= ''+str(tS.numerator)+'/'+str(tS.denominator)
        df_key.loc[i]=[index,name, key, time_signature]



df_key

,id,name,key,time_signature
0,0,Muiñeira dos Fosados,C major,6/8
1,1,Detrás das airas,C major,6/8
2,2,Muiñeira de Quindous,C major,6/8
3,3,Muiñeira de Quindous,C major,6/8
4,4,Muiñeira,C major,6/8
...,...,...,...,...
448,448,Carballesa,C major,2/4
449,449,Carballesa do Incio,C major,2/4
450,450,Muiñeira de Vilarmide,C major,6/8
451,451,Na Ruada,C major,2/4


## Key
We will use music21's method to get the key of each score and save it to our dataframe.

In [6]:
df_key_located=pd.DataFrame(columns=['id','name','parroquia','concellu','province','key','time_signature'])
i=0
for i in range(258):
    #get index of df_located
    index=df_located.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
        key=s.analyze('key')
        #Get name of the song in df in this case we can use the index because the index of df is the same as the id
        name=df_located.loc[i]['name']
        parroquia=df_located.loc[i]['parroquia']
        concellu=df_located.loc[i]['concellu']
        province=df_located.loc[i]['province']
            
        measures0 = s.getElementsByClass(stream.Measure).stream() #numero de compases
        tS = measures0[0].getElementsByClass(meter.TimeSignature).stream()[0]
        time_signature= ''+str(tS.numerator)+'/'+str(tS.denominator)
        df_key_located.loc[i]=[index,name,parroquia,concellu,province, key,time_signature]


df_key_located

,id,name,parroquia,concellu,province,key,time_signature
0,1,Detrás das airas,NaN,As Airas,Ourense,C major,6/8
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8
3,5,Muiñeira de Monforte,NaN,Monforte,Lugo,C major,6/8
4,6,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra,C major,6/8
...,...,...,...,...,...,...,...
253,446,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra,C major,6/8
254,449,Carballesa do Incio,NaN,O Incio,Lugo,C major,2/4
255,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo,C major,6/8
256,452,Danza Anterga de Lugo,NaN,Lugo,Lugo,C major,2/4


## Quarter to Eighth ratio
This ratio is calculated iterating through the whole score and adding incrementing different counters each time a quarter or eighth note is found.

In [7]:
def ratio_negras_corcheas(index):
    s=load_musicxml(index)
    scoreParts = s.parts.stream()
    my_score = scoreParts[0]
    #print all notes in the score
    measure_list= my_score.getElementsByClass(stream.Measure).stream() 

    #Print all notes
    count_negras=0
    count_corcheas=0
    for measure in measure_list:
        for note in measure.notesAndRests:
            #print(note)
            if not note.isRest:
                
                #Print if the note is a crotchet
                if note.duration.type=='eighth':
                    count_corcheas=count_corcheas+1
                if note.duration.type=='quarter':
                    count_negras=count_negras+1
                #print(note.nameWithOctave)

    #print('Negras: ',count_negras)
    #print('Corcheas: ',count_corcheas)
    #Print count_negras to count_corcheas ratio
    #print('Ratio: ',count_negras/count_corcheas)
    return count_negras/count_corcheas
list=[]
for int in range(453):
    val= ratio_negras_corcheas(int)
    #Add it to the dataframe
    list.append(val)
df_key['ratio_corcheas_negras']=list
df_key




,id,name,key,time_signature,ratio_corcheas_negras
0,0,Muiñeira dos Fosados,C major,6/8,0.096154
1,1,Detrás das airas,C major,6/8,0.246575
2,2,Muiñeira de Quindous,C major,6/8,0.312500
3,3,Muiñeira de Quindous,C major,6/8,0.307692
4,4,Muiñeira,C major,6/8,0.180000
...,...,...,...,...,...
448,448,Carballesa,C major,2/4,0.290323
449,449,Carballesa do Incio,C major,2/4,0.126984
450,450,Muiñeira de Vilarmide,C major,6/8,0.092025
451,451,Na Ruada,C major,2/4,0.160714


In [8]:

#Now lets do the same for the located songs
list=[]
for int in range(258):
    #get id of df_located
    index=df_key_located.loc[int]['id']

    val= ratio_negras_corcheas(index)
    #Add it to the dataframe
    list.append(val)

df_key_located['ratio_negras_corcheas']=list
df_key_located

,id,name,parroquia,concellu,province,key,time_signature,ratio_negras_corcheas
0,1,Detrás das airas,NaN,As Airas,Ourense,C major,6/8,0.246575
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8,0.312500
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8,0.307692
3,5,Muiñeira de Monforte,NaN,Monforte,Lugo,C major,6/8,0.240000
4,6,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra,C major,6/8,0.160000
...,...,...,...,...,...,...,...,...
253,446,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra,C major,6/8,0.388889
254,449,Carballesa do Incio,NaN,O Incio,Lugo,C major,2/4,0.126984
255,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo,C major,6/8,0.092025
256,452,Danza Anterga de Lugo,NaN,Lugo,Lugo,C major,2/4,0.181818


In [9]:
#Testing! get from df_key_located the one with index 59
name=df_key_located.loc[59]['name']
#print name
print(name)

Muiñeira de Aldán


## Get Muiñeira class
Since we want to classify them in two different types given its structure:
            C1 ==========> Question 1 == Question 2; Answer 1 != Answer2
            C2 ==========> Question 1 != Question 2; Answer 1 != Answer2
We will do this easily with music21, getting the first measures in which both the first and second question and answer are done and compare them as note, duration tuples to see if they match or not.

In [10]:
import music21
#Method that gets two tuples and returns true if they are equal
def compare_tuples(t1,t2):
    #if different lengths return false
    if len(t1) != len(t2):
        return False
    #now compare each element
    for i in range(len(t1)):
        if t1[i] != t2[i]:
            return False
    return True
#Iterate through located dataframe and print note names
def get_class(s):
    pregunta_1=[]
    respuesta_1=[]
    pregunta_2=[]
    respuesta_2=[]
    count=0
    if s is not None:     
        measures0 = s.getElementsByClass(stream.Measure).stream() #numero de compases
        for measure in measures0:
            if count >=1 and count <= 2:
                for note in measure.notesAndRests:
                    try:
                        if not note.isRest:
                            pregunta_1.append((note.nameWithOctave,note.duration.type))
                    except:
                        print('There seems to be a problem here: ', index)
            if count >=3 and count <= 4:
                for note in measure.notesAndRests:
                    try:
                        if not note.isRest:
                            respuesta_1.append((note.nameWithOctave,note.duration.type))
                    except:
                        print('There seems to be a problem here: ', index)
            if count >=5 and count <= 6:
                for note in measure.notesAndRests:
                    try:
                        if not note.isRest:
                            pregunta_2.append((note.nameWithOctave,note.duration.type))
                    except:
                        print('There seems to be a problem here: ', index)
            if count >=7 and count <= 8:
                for note in measure.notesAndRests:
                    try:
                        if not note.isRest:
                            respuesta_2.append((note.nameWithOctave,note.duration.type))
                    except:
                        print('There seems to be a problem here: ', index)
            if count >=9:
                break
            count+=1
        compare_pregunta=compare_tuples(pregunta_1,pregunta_2)
        compare_respuesta=compare_tuples(respuesta_1,respuesta_2)
        if compare_pregunta and not compare_respuesta:
            return 'C1'
        if not compare_pregunta and not compare_respuesta:
            return 'C2'
            #Type of Muiñeira:
            #C1 ==========> P1==P2; R1!=R2
            #C2 ==========> P1!=P2; R1!=R2


#Iterate through located dataframe and print names
for i in range(258):
    #get index of df_located
    index=df_located.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
        muiñeira_type=get_class(s)
        #Add it to the dataframe
        df_key_located.loc[i,'muiñeira_type']=muiñeira_type

#same but for the whole dataset
for i in range(453):
    #get index of df_located
    index=df_key.loc[i]['id']
    s = load_musicxml(index)
    if s is not None:
        scoreParts = s.parts.stream()
        s = scoreParts[0]
        muiñeira_type=get_class(s)
        #Add it to the dataframe
        df_key.loc[i,'muiñeira_type']=muiñeira_type
df_key

#The 43 score seems to be problematic?

There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems to be a problem here:  43
There seems 

,id,name,key,time_signature,ratio_corcheas_negras,muiñeira_type
0,0,Muiñeira dos Fosados,C major,6/8,0.096154,C2
1,1,Detrás das airas,C major,6/8,0.246575,C1
2,2,Muiñeira de Quindous,C major,6/8,0.312500,C1
3,3,Muiñeira de Quindous,C major,6/8,0.307692,C1
4,4,Muiñeira,C major,6/8,0.180000,C1
...,...,...,...,...,...,...
448,448,Carballesa,C major,2/4,0.290323,C2
449,449,Carballesa do Incio,C major,2/4,0.126984,C1
450,450,Muiñeira de Vilarmide,C major,6/8,0.092025,C2
451,451,Na Ruada,C major,2/4,0.160714,C2


In [11]:
df_key_located


,id,name,parroquia,concellu,province,key,time_signature,ratio_negras_corcheas,muiñeira_type
0,1,Detrás das airas,NaN,As Airas,Ourense,C major,6/8,0.246575,C1
1,2,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8,0.312500,C1
2,3,Muiñeira de Quindous,Quindous,Cervantes,Lugo,C major,6/8,0.307692,C1
3,5,Muiñeira de Monforte,NaN,Monforte,Lugo,C major,6/8,0.240000,C1
4,6,Muiñeira de Ventosa,NaN,Ventosa,Pontevedra,C major,6/8,0.160000,C1
...,...,...,...,...,...,...,...,...,...
253,446,Ribeirana de Pontevedra,NaN,Pontevedra,Pontevedra,C major,6/8,0.388889,C2
254,449,Carballesa do Incio,NaN,O Incio,Lugo,C major,2/4,0.126984,C1
255,450,Muiñeira de Vilarmide,Vilarmide,A Pontenova,Lugo,C major,6/8,0.092025,C2
256,452,Danza Anterga de Lugo,NaN,Lugo,Lugo,C major,2/4,0.181818,C1


In [12]:
print("Whole dataset:")
print(len(df_key[df_key['muiñeira_type']=='C1']))
print(len(df_key[df_key['muiñeira_type']=='C2']))
#print number of rows with muiñeira type C1
print("Located dataset:")
print(len(df_key_located[df_key_located['muiñeira_type']=='C1']))
#print number of rows with muiñeira type C2
print(len(df_key_located[df_key_located['muiñeira_type']=='C2']))


Whole dataset:
268
179
Located dataset:
146
109


In [13]:
#Let's save both dataframes to csv files
df_key.to_csv('data.csv',index=False)
df_key_located.to_csv('data_located.csv',index=False)



